### Import libraries

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import json

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
                                                                                                                                                                                          
from bayes_opt.event import Events
from Utils import newBayesianOptimization, newJSONLogger

In [2]:
# modelos
from sklearn.svm import SVC as SVC

### Import data

In [4]:
data_path = Path().resolve().parent / "magic04.data"
data = pd.read_csv(data_path, header=None)
data.columns=['fLength','fWidth','fSize','fConc','fConc1','fAsym','fM3Long','fM3Trans','fAlpha','fDist','class']

In [6]:
data2 = (
    data
    .assign(
        **{"class": lambda x: x['class'].map({'g': 1, 'h': 0}).astype('int8')}
    )
)

In [7]:
# train test split
X_train, X_test, Y_train, Y_test = train_test_split(
    data2.drop(columns=['class']),
    data2[['class']],
    train_size=0.7,
    random_state=0
)

# scaler
scaler = StandardScaler()
scaler.set_output(transform="pandas")

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [32]:
def opt_bas(C, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C,
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVC(**hyperparameters).fit(X_train, Y_train.values.ravel())

    Y_pred = model.predict(X_test)
    
    return accuracy_score(Y_test, Y_pred)

### Run optimization

In [46]:
# Bounded region of parameter space
pbounds = {'C': (0.01, 500), 'gamma': (0.02, 0.25)}

# Bayes optimizer instantiation
optimizer = newBayesianOptimization(
    f=opt_bas, 
    pbounds=pbounds, 
    random_state=1, 
    verbose=2, 
)

# keep data
log_path = Path().resolve() / "Logs" / "svc.jsonl"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [47]:
# optimizer.res

In [44]:
# get all the current parameters of the model
optimizer.load_previous(log_path)

In [ ]:
optimizer.maximize(init_points=6, n_iter=500)